In [2]:
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import datasets,transforms
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from torch import optim
import time
import util_fei

In [3]:
def conv_block(input_channels, num_channels):
    return nn.Sequential(
        nn.BatchNorm2d(input_channels), nn.ReLU(),
        nn.Conv2d(input_channels, num_channels, 
                    kernel_size=3, padding=1)
    )

In [6]:
class DenseBlock(nn.Module):
    def __init__(self, num_convs, input_channels, num_channels):
        super(DenseBlock, self).__init__()
        layer = []
        for i in range(num_convs):
            layer.append(conv_block(
                num_channels * i + input_channels, num_channels))
        self.net = nn.Sequential(*layer)
    
    def forward(self, X):
        for blk in self.net:
            Y = blk(X)
            X = torch.cat((X, Y), dim=1)
        return X


In [7]:
blk = DenseBlock(2, 3, 10)
X = torch.randn(4, 3, 8, 8)
Y = blk(X)
print(Y.shape)

torch.Size([4, 23, 8, 8])


In [8]:
def transition_block(input_channels, num_channels):
    return nn.Sequential(
        nn.BatchNorm2d(input_channels),
        nn.ReLU(),
        nn.Conv2d(input_channels, num_channels, kernel_size=1),
        nn.AvgPool2d(kernel_size=2, stride=2)
    )

In [9]:
blk = transition_block(23, 10)
print(blk(Y).shape)

torch.Size([4, 10, 4, 4])


In [11]:
# DenseNet 模型
b1 = nn.Sequential(
    nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
)

In [17]:
num_channels, growth_rate = 64, 32
num_convs_in_dense_blocks = [4, 4, 4, 4]
blks = []

for i, num_convs in enumerate(num_convs_in_dense_blocks):
    blks.append(DenseBlock(num_convs, num_channels, growth_rate))

    num_channels += num_convs * growth_rate

    if i != len(num_convs_in_dense_blocks) - 1:
        blks.append(transition_block(num_channels, num_channels // 2))
        num_channels = num_channels // 2



In [18]:
net = nn.Sequential(
    b1,
    *blks,
    nn.BatchNorm2d(num_channels),
    nn.ReLU(),
    nn.AdaptiveAvgPool2d((1,1)),
    nn.Flatten(),
    nn.Linear(num_channels, 10)
)

In [14]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=False,
    transform= transforms.Compose([transforms.Resize(96),
                                transforms.ToTensor()])
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=False,
    transform=transforms.Compose([
        transforms.Resize(96),
        transforms.ToTensor()])
)

batch_size = 256

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


cuda:0


In [19]:
creterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1)

epoch_nums = 10

util_fei.train_model(net, train_dataloader, test_dataloader,creterion, optimizer,device, epoch_num=epoch_nums)

/data/study/git/pytorch_cook/03_CNN/util_fei.py:26: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(m.weight)


training on  cuda:0


/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448265233/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


epoch: 1,	                train_loss: 0.525589,	                train_correct:0.813600,	                test_correct:0.829200,	                test time:31.063614
epoch: 2,	                train_loss: 0.304202,	                train_correct:0.889333,	                test_correct:0.852500,	                test time:27.301594
epoch: 3,	                train_loss: 0.253136,	                train_correct:0.907800,	                test_correct:0.883700,	                test time:27.234114
epoch: 4,	                train_loss: 0.221616,	                train_correct:0.919167,	                test_correct:0.854800,	                test time:27.304331
epoch: 5,	                train_loss: 0.198567,	                train_correct:0.927883,	                test_correct:0.875300,	                test time:27.261773
epoch: 6,	                train_loss: 0.178641,	                train_correct:0.934850,	                test_correct:0.883100,	                test time:27.329575
epoch: 7,	            